In [ ]:
#%%
import requests
import os
import json
import datetime
from tkinter.ttk import Notebook
from tkinter import *
from tkinter import Tk, Canvas
import tkinter.ttk as ttk
from random import randrange
from PIL import Image, ImageTk
from tkinter.scrolledtext import ScrolledText
from tkinter.messagebox import showinfo, askokcancel
from tkinter.filedialog import askopenfilename, asksaveasfilename, asksaveasfile
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import tkinter.messagebox


source = " https://pomber.github.io/covid19/timeseries.json"
r = requests.get(source)

json_data = r.json()
country_list = [country for country in json_data.keys()]
country_list.sort()

class Country:
    def __init__(self, name, data):
        self.name = name
        self.data = data
        self.fechas = self.fechas()
        self.confirmed = self.confirmados()
        self.deaths = self.fallecidos()
        self.recovered = self.recuperados()
        
    def fechas(self):
        datos = [i.get('date') for i in self.data.get(self.name)]           
        return [datetime.datetime.strptime(i, "%Y-%m-%d").date() for i in datos]       
    
    def confirmados(self):
        return [i.get('confirmed') for i in self.data.get(self.name)]
    
    def fallecidos(self):
        return [i.get('deaths') for i in self.data.get(self.name)]
    
    def recuperados(self):
        return [i.get('recovered') for i in self.data.get(self.name)]
    

class App:
    def __init__(self, master):
        self.master = master
        self.master.title("COVID-19 Data Dashboard")
        self.master.resizable(0, 0)
        
        self.master.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.master.resizable(0,0)
        
        
        self.variable = StringVar()
        self.variable.set('')
        self.vista = StringVar()
        self.vista.set('')
        self.valor = IntVar()
        
        #------------------ FRAMES---------------------------
        frm = Frame(master)
        frm1 = Frame(frm)
        frm2 = Frame(frm)
        frm_Paises = LabelFrame(frm1, text="Lista de Paises")
        frm_selected = LabelFrame(frm1, text="Paises Seleccionados")
        frm_graph = LabelFrame(frm2, text="Matplotlib Data Graph")
        frm_info = LabelFrame(frm2, text="Informacion del gráfico")
        
        frm.pack()
        frm1.pack(side='left', padx=5, pady=5, anchor='n')
        frm2.pack(side='left', padx=5, pady=5, anchor='n')
        frm_Paises.pack(fill='both', padx=10)
        frm_selected.pack(fill='both', padx=10, pady=50, anchor='s')
        frm_graph.pack()
        frm_info.pack(side='left', expand=True, fill='x')
        
        #-------------- WIDGETS --------------------------
        self.state_bar = Label(master, text="Datos actualizados de https://pomber.github.io/covid19/timeseries.json", bd=1, relief=SUNKEN)
        self.state_bar.pack(side='bottom', expand=True, fill='x')
        
        self.scrollbar = Scrollbar(frm_Paises)
        self.lista_paises = Listbox(frm_Paises)
        
        for i in country_list:
            self.lista_paises.insert(END, i)
        
        self.lista_paises.config( yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.lista_paises.yview)
        
        self.lista_paises.pack(side='left', fill='y', padx=5, pady=5)
        self.scrollbar.pack(side='left', fill='y')
        
        self.scrollbar2 = Scrollbar(frm_selected, orient=VERTICAL)
        self.paises_seleccionados = Listbox(frm_selected, yscrollcommand=self.scrollbar2.set)
        
        self.scrollbar2.config(command=self.paises_seleccionados.yview)
        
        self.paises_seleccionados.pack(side='left', fill='y')
        self.scrollbar2.pack(side='left', fill='y')
        
        self.fig, self.ax = plt.subplots(figsize=(14, 6))
        self.fig.set_facecolor("#F0F0F0")
        self.grafica = FigureCanvasTkAgg(self.fig, master=frm_graph)
        self.grafica.get_tk_widget().pack(side='top')
        
        self.herramientas = NavigationToolbar2Tk(self.grafica, frm_graph)
        self.herramientas.update()
        
        self.ax.set_title("COVID-19 Country Data")
        self.ax.cla()
        
        self.rdoConfirmado = Radiobutton(frm_info, text="Confirmados", variable=self.valor, value=1, state=DISABLED, command=self.select_info)
        self.rdoFallecidos = Radiobutton(frm_info, text="Fallecidos", variable=self.valor, value=2, state=DISABLED, command=self.select_info) 
        self.rdoRecuperados = Radiobutton(frm_info, text="Recuperados", variable=self.valor, value=3, state=DISABLED, command=self.select_info)
        self.btSalir = Button(frm2, text="Salir", command=self.salir, width=10)
        self.rdoConfirmado.select()
        
        self.rdoConfirmado.grid(row=0, column=0, padx=70, pady=15, sticky='w')
        self.rdoFallecidos.grid(row=0, column=1, padx=200, pady=15, sticky='w')
        self.rdoRecuperados.grid(row=0, column=2, padx=5, pady=15, sticky='w')
        self.btSalir.pack(side='right', padx=10, pady=10)
        
        self.lista_paises.bind('<Double-Button-1>', self.select_country)
        self.paises_seleccionados.bind('<Double-Button-1>', self.deselect)
        frm_Paises.bind('<Enter>', self.state_country)
        frm_selected.bind('<Enter>', self.delete_country)
        self.rdoConfirmado.bind('<Enter>', self.state_conf)
        self.rdoFallecidos.bind('<Enter>', self.state_fall)
        self.rdoRecuperados.bind('<Enter>', self.state_rec)
        self.btSalir.bind("<Enter>", self.on_enter0)
        
        self.lista_seleccion=[]
        self.y1=[]
        self.y2=[]
        self.y3=[]
        self.lista_conf=[]
        self.lista_fall=[]
        self.lista_rec=[]
        self.lista_n=[]
        
    def select_country(self, event):
        self.rdoConfirmado.config(state='normal')
        self.rdoFallecidos.config(state='normal')
        self.rdoRecuperados.config(state='normal')
            
        l1 = self.lista_paises.curselection()
        pos1 = l1[0]
        nam1 = self.lista_paises.get(pos1)
        
        seleccion=country_list.pop(pos1)
        self.lista_seleccion.append(seleccion)
        print(self.lista_seleccion)
        self.lista_paises.delete(0, END)
        
        for i in country_list:
            self.lista_paises.insert(END, i)
        
        self.paises_seleccionados.delete(0, END)
        
        for i in self.lista_seleccion:
            self.paises_seleccionados.insert(END, i)
            
        pais = Country(nam1, json_data)
      
        self.ax.cla() 
        self.y1.append(pais.confirmed[len(pais.confirmed)-1])
        self.y2.append(pais.deaths[len(pais.deaths)-1])
        self.y3.append(pais.recovered[len(pais.recovered)-1])
        self.x_max=pais.fechas
        ym1=max(self.y1)
        ym2=max(self.y2)
        ym3=max(self.y3)
        
        self.lista_conf.append(pais.confirmed)
        self.lista_fall.append(pais.deaths)
        self.lista_rec.append(pais.recovered)
        self.lista_n.append(nam1)
            
        if self.valor.get()==1:
            self.ax.cla() 
            for i in range(len(self.lista_conf)):
                self.ax.plot(self.x_max, self.lista_conf[i], '-o',label=self.lista_n[i])                
                self.ax.grid(linestyle='--')                   
                self.ax.set_title("COVID-19 Country Data")
                self.ax.set_xlabel("Fechas")
                self.ax.set_ylabel("Numero de Personas")                
                self.ax.legend()                
                self.grafica.draw()
                
        elif self.valor.get()==2:
            self.ax.cla() 
            for i in range(len(self.lista_fall)):
                self.ax.plot(self.x_max, self.lista_fall[i], '-o',label=self.lista_n[i])                                
                self.ax.grid(linestyle='--')                   
                self.ax.set_title("COVID-19 Country Data")
                self.ax.set_xlabel("Fechas")
                self.ax.set_ylabel("Numero de Personas")                
                self.ax.legend()              
                self.grafica.draw()
            
        elif self.valor.get()==3:
            self.ax.cla() 
            for i in range(len(self.lista_fall)):
                self.ax.plot(self.x_max, self.lista_rec[i], '-o',label=self.lista_n[i])                                
                self.ax.grid(linestyle='--')                   
                self.ax.set_title("COVID-19 Country Data")
                self.ax.set_xlabel("Fechas")
                self.ax.set_ylabel("Numero de Personas")                
                self.ax.legend()                
                self.grafica.draw()
    
    def deselect(self, event):
        l2 = self.paises_seleccionados.curselection()
        pos2 = l2[0]
        nam2 = self.paises_seleccionados.get(pos2)
        
        deselection = self.lista_seleccion.pop(pos2)
        country_list.append(deselection)
        country_list.sort()
        self.lista_paises.delete(0, END)
        self.paises_seleccionados.delete(0, END)
        
        for i in country_list:
            self.lista_paises.insert(END, i)
        
        self.paises_seleccionados.delete(0, END)
        for i in self.lista_seleccion:
            self.paises_seleccionados.insert(END, i)
            
        pais = Country(nam2, json_data)
        
   
        self.lista_conf.pop(self.lista_conf.index(pais.confirmed))
        self.lista_fall.pop(self.lista_fall.index(pais.deaths))
        self.lista_rec.pop(self.lista_rec.index(pais.recovered))
        self.lista_n.pop(self.lista_n.index(nam2))
        
        if self.lista_seleccion == []:
            self.rdoConfirmado.config(state='disabled')
            self.rdoFallecidos.config(state='disabled')
            self.rdoRecuperados.config(state='disabled')
            self.ax.clear()
            self.grafica.draw()
            
           
        else:
            if self.valor.get()==1:
                self.ax.cla() 
                for i in range(len(self.lista_conf)):
                    self.ax.plot(self.x_max, self.lista_conf[i], '-o',label=self.lista_n[i])                                
                    self.ax.grid(linestyle='--')                   
                    self.ax.set_title("COVID-19 Country Data")
                    self.ax.set_xlabel("Fechas")
                    self.ax.set_ylabel("Numero de Personas")
                    self.ax.legend()              
                    self.grafica.draw()
                    
            elif self.valor.get()==2:
                self.ax.cla() 
                for i in range(len(self.fall)):
                    self.ax.plot(self.x_max, self.lista_fall[i], '-o',label=self.lista_n[i])
                    self.ax.grid(linestyle='--')                   
                    self.ax.set_title("COVID-19 Country Data")
                    self.ax.set_xlabel("Fechas")
                    self.ax.set_ylabel("Numero de Personas")
                    self.ax.legend()               
                    self.grafica.draw()
            
            elif self.valor.get()==3:
                self.ax.cla() 
                for i in range(len(self.lista_fall)):
                    self.ax.plot(self.x_max, self.recup[i], '-o',label=self.lista_n[i])                
                    self.ax.grid(linestyle='--')                   
                    self.ax.set_title("COVID-19 Country Data")
                    self.ax.set_xlabel("Fechas")
                    self.ax.set_ylabel("Numero de Personas")
                    self.ax.legend()
                    self.grafica.draw()
    
    def select_info(self):
        if self.valor.get()==1:
            self.ax.cla()
            print(len(self.lista))
            print(self.lista_conf)
            for i in range(len(self.lista_conf)):
                self.ax.plot(self.x_max, self.lista_conf[i], '-o' ,label=self.lista_n[i])
                self.ax.grid(linestyle='--')
                
                self.ax.set_title("COVID-19 Country Data")
                self.ax.set_xlabel("Fechas")
                self.ax.set_ylabel("Numero de Personas")
                self.ax.legend()
                self.grafica.draw()
                
        elif self.valor.get()==2:
            self.ax.cla()
            for i in range(len(self.lista_fall)):
                self.ax.plot(self.x_max, self.lista_fall[i], '-o',label=self.lista_n[i])
                self.ax.grid(linestyle='--')
            
                self.ax.grid(linestyle='--')                   
                self.ax.set_title("COVID-19 Country Data")
                self.ax.set_xlabel("Fechas")
                self.ax.set_ylabel("Numero de Personas")
                self.ax.legend()
                self.grafica.draw()
        
        elif self.valor.get()==3:
            self.ax.cla() 
            for i in range(len(self.lista_fall)):
                self.ax.plot(self.x_max, self.lista_rec[i], '-o',label=self.lista_n[i])                
                self.ax.grid(linestyle='--')                   
                self.ax.set_title("COVID-19 Country Data")
                self.ax.set_xlabel("Fechas")
                self.ax.set_ylabel("Numero de Personas")                
                self.ax.legend()
                self.grafica.draw()
                
    def state_conf(self, event):
        self.state_bar.config(text="Grafica de casos confirmados")
    
    def state_fall(self, event):
        self.state_bar.config(text="Grafica de pacientes fallecidos")
    
    def state_rec(self, event):
        self.state_bar.config(text="Grafica de pacientes recuperados")
        
    def delete_country(self, event):
        self.state_bar.config(text="Elimine un pais seleccionado")    
                
    def state_country(self, event):
        self.state_bar.config(text="Seleccine un país") 
    def on_enter0(self,event):
        self.state_bar.config(text="Cerrar el programa")
           
                    
    def on_closing(self):
            if messagebox.askokcancel("Salir", "¿Desea salir de la aplicación?"):
                root.destroy()
    
    def salir(self):
        yes = tkinter.messagebox.askyesno("Salir", "¿Desea salir de la aplicación?")
        if yes:
            root.destroy()
                    

root = Tk()
app = App(root)
root.mainloop()

['Australia']


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-033b69d1ed50>", line 342, in on_closing
    if messagebox.askokcancel("Salir", "¿Desea salir de la aplicación?"):
NameError: name 'messagebox' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-033b69d1ed50>", line 342, in on_closing
    if messagebox.askokcancel("Salir", "¿Desea salir de la aplicación?"):
NameError: name 'messagebox' is not defined
